In [ ]:
!pip install virtualenv
!virtualenv myenv
!source myenv/bin/activate
!pip install openai
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 13.9 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 2531ms
  creator CPython3Posix(dest=/content/myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.2, setuptools==75.1.0, wheel==0.44.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

In [ ]:
from google.colab import userdata
OPENAI_API_KEY= userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

### **Long Audio**

In [ ]:
# !pip install pydub
"""This Fuction will segment audio in 10 mins each and store it in a list"""
from pydub import AudioSegment
song = AudioSegment.from_mp3("test_long_audio.mp3")
ten_minutes = 10 * 60 * 1000
segments = []
for i in range(0, len(song), ten_minutes):
    segment = song[i:i + ten_minutes]
    segments.append(segment)

In [ ]:
!pip install openai
from openai import OpenAI
from dotenv import load_dotenv
import os
from google.colab import userdata
OPENAI_API_KEY= userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
full_transcription = ""

for idx, segment in enumerate(segments):
    segment.export(f"segment_{idx}.mp3", format="mp3")

    with open(f"segment_{idx}.mp3", "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            response_format="text"
        )
        full_transcription += '. '+ transcription

In [ ]:
with open("transcription.txt", "w") as f:
    f.write(full_transcription)
print("Transcription saved to transcription.txt")

Transcription saved to transcription.txt


In [ ]:
with open("transcription.txt", "r") as file:
    transcription_content = file.read()
print(transcription_content)

. UCERA, Y-O-S-A-R-A, knee news, N-U-N-E-Z in the city of Divinitka. They were going straight when another car cut them off from the opposite direction. Airbag deployed, caused bruises about over the chest, left chest, the left breast, and then it's about 40 centimeter bruises over left thigh and about 30 centimeter over outer leg, and then over the shin as well, and the bruises on the left leg and cut is present over the inner, the top of the knee and inner legs. All right, where do you have the pain? You have a chest pain? Yeah. How often do you have a chest pain? Um, it's pretty much when, well, it's always there, but it's pretty much when I, if I lay on this side, if I lean, lean over on that side. How about tender and touch? Yeah. What would you rate it? Right now or in the beginning? Just right now. Right now, it's like a, like a three. Three? Yeah, when I touch, it's still really sore. Tenderness over the left breast is achy, is under, mid, and constant, rated three. It's gettin

In [ ]:
import re

def preprocess_transcription(transcription):
    filler_words = r'\b(um|uh|you know|like|so|anyway|mmm)\b'
    transcription = re.sub(filler_words, '', transcription, flags=re.IGNORECASE)
    transcription = re.sub(r'\.{2,}', '.', transcription)

    transcription = re.sub(r'\s+', ' ', transcription).strip()

    keywords = ["pain", "hurt", "shoulder", "knee", "chest", "hospital", "accident", "Tylenol"]
    for keyword in keywords:
        transcription = re.sub(f'(?i)({keyword})', r'\n\1', transcription)

    transcription = transcription.replace("the first day", "the initial day")

    if transcription:
        transcription = transcription[0].upper() + transcription[1:]
    return transcription

preprocess_transcription(transcription_content)

". UCERA, Y-O-S-A-R-A, \nknee news, N-U-N-E-Z in the city of Divinitka. They were going straight when another car cut them off from the opposite direction. Airbag deployed, caused bruises about over the \nchest, left \nchest, the left breast, and then it's about 40 centimeter bruises over left thigh and about 30 centimeter over outer leg, and then over the shin as well, and the bruises on the left leg and cut is present over the inner, the top of the \nknee and inner legs. All right, where do you have the \npain? You have a \nchest \npain? Yeah. How often do you have a \nchest \npain? , it's pretty much when, well, it's always there, but it's pretty much when I, if I lay on this side, if I lean, lean over on that side. How about tender and touch? Yeah. What would you rate it? Right now or in the beginning? Just right now. Right now, it's a, a three. Three? Yeah, when I touch, it's still really sore. Tenderness over the left breast is achy, is under, mid, and constant, rated three. It's

In [ ]:
!pip install -q -U google-generativeai
import google.generativeai as genai
from google.colab import userdata
genai.configure(api_key= userdata.get('GEMINI_API_KEY'))
model = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
example_form = """
<h1>Medical Report</h1><h2>Patient Information</h2><ul><li><strong>Name:</strong> Emilio Ferruggia</li><li><strong>Date of Accident:</strong> July 5th</li><li><strong>Location of Accident:</strong> Thousand Oaks</li></ul><h2>Accident Description</h2><p>Emilio Ferruggia was driving straight with the flow of traffic when another driver turned left from a middle section and hit his driver door. The impact also affected the front fender. Emilio did not lose consciousness and did not call the police or an ambulance. He was shaken up but did not seek immediate medical attention.</p><h2>Initial Medical Consultation</h2><ul><li><strong>First Doctor Seen:</strong> Dr. Khalili</li><li><strong>Prescribed Medication:</strong> Unknown</li></ul><h2>Symptoms and Pain Assessment</h2><h2>Low Back Pain</h2><ul><li><strong>Location:</strong> Low back</li><li><strong>Frequency:</strong> Whenever lifting something heavy</li><li><strong>Type of Pain:</strong> Numbness, occasionally sharp</li><li><strong>Pain Radiation:</strong> Does not radiate to the leg</li><li><strong>Pain Level:</strong> 7 out of 10</li><li><strong>Aggravating Factors:</strong> Working, lifting, walking, running</li><li><strong>Alleviating Factors:</strong> Medication, elevating, relaxing</li></ul><h2>Neck Pain</h2><ul><li><strong>Frequency:</strong> Comes and goes</li><li><strong>Type of Pain:</strong> Sharp</li><li><strong>Pain Radiation:</strong> Travels to shoulder and arm</li><li><strong>Pain Level:</strong> 7 out of 10</li><li><strong>Aggravating Factors:</strong> Turning and working</li><li><strong>Alleviating Factors:</strong> Stretch, rest, over-the-counter medication</li></ul><h2>Knee Pain</h2><ul><li><strong>Location:</strong> Right knee</li><li><strong>Frequency:</strong> When standing for too long</li><li><strong>Type of Pain:</strong> Achy, swelling</li><li><strong>Pain Level:</strong> 5 out of 10</li><li><strong>Aggravating Factors:</strong> Standing for extended periods</li><li><strong>Alleviating Factors:</strong> Rest, medication, walking</li></ul><h2>Past Health History</h2><ul><li><strong>Surgeries:</strong> <ul><li>[object Object]</li><li>[object Object]</li></ul></li><li><strong>Pain in Previous Surgery Areas:</strong> Knee pain has worsened since the accident</li></ul><h2>Physical Examination</h2><ul><li><strong>Neck Range of Motion:</strong> <ul><li>Forward flexion: 40 degrees</li><li>Backward extension: 40 degrees</li><li>Left rotation: 40 degrees</li><li>Right rotation: 50 degrees</li><li>Right lateral flexion: 20 degrees</li><li>Left lateral flexion: 30 degrees</li></ul></li><li><strong>Back Range of Motion:</strong> <ul><li>Forward flexion: 30 degrees</li><li>Backward extension: 10 degrees</li><li>Left rotation: 20 degrees</li><li>Right rotation: 30 degrees</li><li>Left lateral flexion: 15 degrees</li><li>Right lateral flexion: 10 degrees</li></ul></li><li><strong>Knee Range of Motion:</strong> Full range of motion in the right knee with pain</li><li><strong>Tenderness and Spasms:</strong> <ul><li>Cervical: C2 to C7</li><li>Thoracic: T1 to T5</li><li>Lumbar: L2 to S1</li></ul></li><li><strong>Muscle Testing:</strong> <ul><li>Right lower extremity: 4 out of 5</li><li>Upper extremity: 5 out of 5</li></ul></li><li><strong>Sensation:</strong> Increased sensation in the right lower extremity</li></ul><h2>Treatment Plan</h2><ul><li><strong>Frequency:</strong> Three times a week (Tuesday, Thursday, Friday)</li><li><strong>X-rays:</strong> X-ray of the right knee was performed</li></ul><h2>Additional Notes</h2><p>No problems with hips or shoulders. No numbness or tingling in fingers. Patient cannot touch toes when bending forward.</p>
"""

In [ ]:
response = model.generate_content(f"Generate a doctors form in html tags, Given the transcribed conversation:\n {transcription_content }. use the following example form as reference {example_form}. note: no input is to be take.")
print(response.text)

Extracting form using transcription

In [ ]:
prompt = f"Generate a doctors form in html tags, Given the transcribed conversation:\n {transcription_content }. use the following example form as reference {example_form}. note: no input filed is to be used. just display the information using the transcription i have provided"
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are responsibe for creating doctors form suing the trascription_content provided."},
    {"role": "user", "content": prompt}
  ])

NotFoundError: Error code: 404 - {'error': {'message': 'The model `01-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [ ]:
print(completion.choices[0].message.content)

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Doctor's Report</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
        }
        .section {
            margin-bottom: 20px;
        }
        .section h2 {
            border-bottom: 1px solid #000;
            padding-bottom: 5px;
            margin-bottom: 10px;
        }
        .field {
            margin: 5px 0;
        }
        .label {
            font-weight: bold;
        }
    </style>
</head>
<body>
    <header class="section">
        <h2>Patient Information</h2>
        <div class="field"><span class="label">Name:</span> UCERA YOSARA NUNEZ</div>
        <div class="field"><span class="label">City:</span> Divinitka</div>
    </header>
    
    <section class="section">
        <h2>Accident Details</h2>
        <div class="field">They were going straight when another car cut them off from the opposite direction.</div>
    </section